<a href="https://colab.research.google.com/github/LC1332/Zero-Haruhi/blob/main/notebook/novel_data/%E8%AF%BB%E5%8F%96%E5%B9%B6%E6%89%B9%E9%87%8F%E6%8A%BD%E5%8F%96%E5%8D%A1%E7%89%87.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


这个pkl在

https://drive.google.com/file/d/1LSVaMmKkfuHp1rm3DK0U2zosgZiv34dJ/view?usp=sharing

In [2]:
input_name = "/content/drive/MyDrive/CardBuild/exp0212/card2extract.pkl" # 冷子昂改为你下载后的目录

import pickle

with open(input_name, 'rb') as f:
    datas = pickle.load(f)



In [3]:
jsonl_save_folder = "/content/drive/MyDrive/CardBuild/exp0212/" # 冷子昂改为你要保存的目录

载入openai

In [4]:
!pip install -q openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [5]:
import os
import httpx

# from google.colab import userdata # 冷子昂 不是colab就注释掉

os.environ["OPENAI_API_KEY"]  = ""  # 冷子昂改这个key
os.environ["OPENAI_API_BASE"] = "" # 冷子昂 这个不设置就注释掉

import openai
from openai import AsyncOpenAI

model_name = "gpt-4-0125-preview" # 冷子昂改为 gpt-4-0125-preview

In [6]:
!pip install -q aiofiles tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 17.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.


In [16]:
import os
import json
import time
# import openai
from openai import AsyncOpenAI

if os.getenv("OPENAI_API_KEY") is None:
    aclient = AsyncOpenAI( api_key = os.getenv("OPENAI_API_KEY" ) )
else:
    aclient = AsyncOpenAI( base_url = os.getenv("OPENAI_API_BASE"), api_key = os.getenv("OPENAI_API_KEY") )

import asyncio
import aiofiles
import tiktoken
import hashlib
# from connector import AsyncPGConnector
from tqdm.asyncio import tqdm as tqdm

enc = tiktoken.encoding_for_model("gpt-3.5-turbo")
en2zh_ratio = 2.3

delay = 1
concurrency_limit = 16

max_file_size = 1024**3

In [17]:
async def dealing_messages(messages):
    try:
        # request_token = sum([len(enc.encode(msg['content'])) for msg in messages])
        # response_token = int(len(enc.encode(text)) * en2zh_ratio) + 64

        model = model_name

        resp = await aclient.chat.completions.create(
            model=model,
            messages=messages,
            temperature=0,
            response_format={ "type": "json_object" },
        )

        try:
            result = resp.choices[0].message.content
            result = result.strip()
            return result
        except json.JSONDecodeError:
            print("Invalid json: ", result)
            return None
        except:
            raise Exception(f"Invalid API response: {resp}")

    except Exception as e:
        print(f"[Error] {e}")
        return None


In [18]:
async def getTranslation(item):
    if "messages" not in item:
        return None
    else:
        for i in range(3):
            result = await dealing_messages(item['messages'])
            if result is not None:
                item["response"] = result
                return item
            else:
                continue

    return None

async def process(item, semaphore):
    async with semaphore:
        try:
            output_folder = "/content/output"
            if "output_folder" in item:
                output_folder = item["output_folder"]
            if not os.path.exists(output_folder):
                os.makedirs(output_folder)

            file_path = os.path.join(output_folder, f"{item['id']}.txt")

            if os.path.exists(file_path):
                return

            it = await getTranslation(item)
            if it is None:
                raise Exception(item['id'])

            with open(file_path, 'w', encoding='utf-8') as f:
                json.dump(it, f, ensure_ascii=False)

        except Exception as e:
            print(f"Error processing entry: {e}")

In [19]:
print(datas[0].keys())

dict_keys(['id', 'input', 'response'])


In [20]:
async def main(datas):

    output_folder = "/content/output"

    process_data = []

    for data in datas:
        id = data['id']
        process_data.append({
            "id": id,
            "messages": [{"role":"user","content":data["input"]}],
            "output_folder": output_folder
        })

    tasks = []

    semaphore = asyncio.Semaphore(concurrency_limit)

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # # print(f"Already processed {len(exist_list)} items...")

    # id = set()

    for item in process_data:
        file_path = os.path.join(output_folder, f"{item['id']}.txt")

        if os.path.exists(file_path):
            continue

        tasks.append(asyncio.create_task(process(item, semaphore)))

    async for task in tqdm(tasks, total=len(tasks), desc="Processing items"):
        await task
        time.sleep(delay)

In [ ]:
start_id = 0
end_id = len(datas)

current_tasks = datas[start_id:end_id]

await main(current_tasks)

temp_output_folder = "/content/output"

for id in range(start_id, end_id):
    id_str = datas[id]["id"]
    file_path = os.path.join(temp_output_folder, f"{id_str}.txt")
    if os.path.exists(file_path):
        with open(file_path, 'r', encoding='utf-8') as f:
            try:
                data = json.load(f)
                response = data["response"]
            except:
                continue

        if response is not None:
            datas[id]["response"] = response


final_save_name = jsonl_save_folder + str(start_id) + "_to_" + str(end_id) + ".txt"

with open(final_save_name, 'w', encoding='utf-8') as f:
    for id in range(start_id, end_id):
        json.dump(datas[id], f, ensure_ascii=False)
        f.write('\n')

In [13]:
print(datas[3]['response'])

{
  "analysis": {
    "人物设定": "霍雨浩是一个具有特殊能力的角色，他拥有极致之冰魂力和魂灵伙伴雪女，同时还有九级炼丹炉和亡灵半位面等神器，是一个非常重要的角色",
    "人物特点": "他聪明、机智，善于利用自己的能力和工具解决问题，同时对自己的伙伴和朋友非常关心",
    "语言风格": "他的语言风格比较成熟稳重，但在面对朋友和伙伴时也会展现出幽默和温暖的一面"
  },
  "most_related_items": {
    "经历": "霍雨浩的经历包括炼丹、使用神器、与魂兽交流等",
    "身份": "他是一个拥有特殊能力和神器的角色，在故事中扮演着重要的角色",
    "性格特点": "他聪明、机智，善于利用自己的能力和工具解决问题，同时对自己的伙伴和朋友非常关心",
    "经典台词": "['第一次遇到', '好奇异啊！', '渐渐感到有些不对了', '开启', '现在让我们开始']",
    "恋爱状态": "对萧炎有特殊好感"
  },
  "claim": "我将从dialogue中总结角色的经历, 身份, 性格特点, 经典台词, 恋爱状态信息",
  "attributes": {
    "经历": "霍雨浩的经历包括炼丹、使用神器、与魂兽交流等",
    "身份": "他是一个拥有特殊能力和神器的角色，在故事中扮演着重要的角色",
    "性格特点": "他聪明、机智，善于利用自己的能力和工具解决问题，同时对自己的伙伴和朋友非常关心",
    "经典台词": "['第一次遇到', '好奇异啊！', '渐渐感到有些不对了', '开启', '现在让我们开始']",
    "恋爱状态": "对萧炎有特殊好感"
  }
}
